In [2]:
import TrainModels 
import ReplaysParser 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_30000.weights" #best model so far
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_30000.weights"
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_loss-stop_30000.weights"
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_loss-stop_10.weights" #benchmark model, basically no training it random guesses at best
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_basic-features_15000.weights" #short model, 42 basic features only
#lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_basic-and-army-features_15000.weights" #basic features + army data model
lstmWeightsPath = "../TrainedModels/lstm_various-match-length_not-normalized_all-features_15000.weights" # all features

lstm = TrainModels.UseTrainedLSTMModel(lstmWeightsPath)

In [8]:
#validationDataFolder = "M:/Uni/sc2eval/SerializedDataForValidation"
hits = np.zeros(26)
confidences = np.zeros(26)
matches = np.zeros(26)

matchesCount = 10
numberOfFeatures = 322


for j in range(matchesCount):
    vectorizedReplayData = ReplaysParser.LoadVectorizedData(1, 15010 + j)
    #vectorizedReplayData = ReplaysParser.ExtractBasicAndArmyData(vectorizedReplayData)
    matchLength = len(vectorizedReplayData[0])
    normalizedData, expectedResults =  TrainModels.PrepareInputData(vectorizedReplayData, 1, matchLength, numberOfFeatures)
    
    for percent in range(26):
        i = round((percent/26)*matchLength)
        if(percent == 25):
            i = matchLength - 1
        input = np.zeros([1,i+1,numberOfFeatures])
        input[0] = normalizedData[0][:i+1]
        expectedOutput = expectedResults[0][0]

        predictions = lstm.predict(input)
        finalPredictions = predictions.argmax(axis=2)[0][-1]
        confidence = max(predictions[0][-1][0],predictions[0][-1][1])

        matches[percent] = matches[percent] + 1
        confidences[percent] = confidences[percent] + confidence
        if(finalPredictions == expectedOutput):
            hits[percent] = hits[percent] + 1

    print("Analyzed matches "+str(j)+"/"+str(matchesCount),end='\r')

plt.plot(hits/matchesCount, label="Accuracy")
plt.plot(confidences/matchesCount, label="Confidence")
plt.legend()
plt.show()

print (hits/matchesCount)
print (confidences/matchesCount)

IndexError: list index out of range